<a href="https://colab.research.google.com/github/biagiominio/telerilevamento_2021/blob/main/MoD_r.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#@title **Install rgee dependencies**
system('sudo add-apt-repository ppa:ubuntugis/ubuntugis-unstable')
system('sudo apt-get update')
system('sudo apt-get install libudunits2-dev libgdal-dev libgeos-dev libproj-dev')
system('sudo apt-get install libprotobuf-dev protobuf-compiler libv8-dev libjq-dev')
install.packages('sf')
install.packages('mapview')
install.packages('cptcity')
install.packages('geojsonio')
remotes::install_github("r-spatial/rgee")

In [ ]:
library(rgee)
ee_Initialize(display = TRUE)

In [ ]:
library(rgee)
library(sf)
library(cptcity)
library(mapview)

Filtro e seleziono il solo poligono "Italy" dalla collezione 'USDOS/LSIB_SIMPLE/2017' per creare la mia ROI


In [ ]:
ita <- ee$FeatureCollection('USDOS/LSIB_SIMPLE/2017')$filter(ee$Filter$eq('country_na', 'Italy'))      

In [ ]:
region <- ita$geometry()$bounds()    

Recupera il dataset MODIS Terra Vegetation Indices 16-Day Global 250m come un ee.ImageCollection e seleziona la banda EVI.

In [ ]:
col <- ee$ImageCollection('MODIS/006/MOD13Q1')$select('EVI')

Raggruppa le immagini per data composta

In [ ]:
col <- col$map(function(img) {
  doy <- ee$Date(img$get('system:time_start'))$getRelative('day', 'year')
  img$set('doy', doy)
})
distinctDOY <- col$filterDate('2010-01-01', '2011-01-01')

In [ ]:
filter <- ee$Filter$equals(leftField = 'doy', rightField = 'doy');

In [ ]:
join <- ee$Join$saveAll('doy_matches')
joinCol <- ee$ImageCollection(join$apply(distinctDOY, col, filter))

In [ ]:
comp <- joinCol$map(function(img) {
  doyCol = ee$ImageCollection$fromImages(
    img$get('doy_matches')
  )
  doyCol$reduce(ee$Reducer$median())
})


In [ ]:
visParams = list(
  min = 0.0,
  max = 9000.0,
  bands = "EVI_median",
  palette = c(
    'FFFFFF', 'CE7E45', 'DF923D', 'F1B555', 'FCD163', '99B718', '74A901',
    '66A000', '529400', '3E8601', '207401', '056201', '004C00', '023B01',
    '012E01', '011D01', '011301'
    )
)

In [ ]:
rgbVis <- comp$map(function(img) {
  do.call(img$visualize, visParams) %>% 
    ee$Image$clip(ita)
})

In [ ]:
gifParams <- list(
  region = region,
  dimensions = 500,
  crs = 'EPSG:3857',
  framesPerSecond = 5
)

In [ ]:
print(rgbVis$getVideoThumbURL(gifParams))
browseURL(rgbVis$getVideoThumbURL(gifParams))

In [ ]:
system("sudo apt-get install libmagick++-dev", intern = TRUE)

In [ ]:
install.packages("magick")

In [ ]:
data_modis <- c('Jan', 'Jan',
                'Feb', 'Feb',
                'Mar', 'Mar',
                'Apr', 'Apr',
                'May', 'May',
                'Jun', 'Jun',
                'Jul', 'Jul',
                'Aug', 'Aug',
                'Sept', 'Sept',
                'Oct',
                'Nov', 'Nov',
                'Dec', 'Dec')

In [ ]:
data_modis <- c('January','January',
                'February','February',
                'March','March',
                'April','April',
                'May','May',
                'June','June',
                'June','June',
                'August','August',
                'September','September',
                'October',
                'November','November',
                'December','December')

In [ ]:
library(magick)
animation <- ee_utils_gif_creator(rgbVis, gifParams, mode = "wb")
animation %>% 
  ee_utils_gif_annotate(
    text = "EVI: MODIS/006/MOD13Q1",
    size = 13, color = "white",
    location = "+10+475"
  ) %>% 
  ee_utils_gif_annotate(
    text = "_____________________", 
    size = 30, 
    location = "+280+30",
    color = "black", 
    font = "arial",
    kerning = 3,
    weight = 400,
    boxcolor = "#000000"
  ) %>% 
  ee_utils_gif_annotate(
    text = data_modis, 
    size = 25, 
    location = "+280+30",
    color = "white", 
    font = "arial",
    kerning = 0,
    weight = 400,
    boxcolor = "#000000"
  ) #-> animation_wtxt  

#ee_utils_gif_save(animation_wtxt, path = "raster_as_ee.gif")

Linking to ImageMagick 6.9.7.4
Enabled features: fontconfig, freetype, fftw, lcms, pango, x11
Disabled features: cairo, ghostscript, heic, raw, rsvg, webp

Using 2 threads

1. Creating gif ... please wait ....

1. Downloading GIF from: https://earthengine.googleapis.com/v1alpha/projects/earthengine-legacy/videoThumbnails/3ff3742e3e0fdbd045ce6239ef1b7210-db363a7aec6c3df22ce21c57eb3acc69:getPixels



# A tibble: 23 x 7
   format width height colorspace matte filesize density
   <chr>  <int>  <int> <chr>      <lgl>    <int> <chr>  
 1 GIF      423    500 sRGB       TRUE         0 72x72  
 2 GIF      422    499 sRGB       TRUE         0 72x72  
 3 GIF      422    499 sRGB       TRUE         0 72x72  
 4 GIF      422    499 sRGB       TRUE         0 72x72  
 5 GIF      422    498 sRGB       TRUE         0 72x72  
 6 GIF      422    499 sRGB       TRUE         0 72x72  
 7 GIF      422    499 sRGB       TRUE         0 72x72  
 8 GIF      421    499 sRGB       TRUE         0 72x72  
 9 GIF      422    500 sRGB       TRUE         0 72x72  
10 GIF      423    500 sRGB       TRUE         0 72x72  
# … with 13 more rows